In [1]:
import pandas as pd
import statistics as sts
from numpy import nan

In [12]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
from Funciones import fold_change

In [3]:
genes1 = pd.read_csv('../Genes/genes-metabolismo.csv', header = None)
genes2 = pd.read_csv('../Genes/genes-metabolismo.csv', header = None)
genes3 = pd.read_excel('../Genes/genes2Nuevo.xlsx',header = None)
genes = set(genes1[0]).union(set(genes2[0])).union(genes3[0])

In [4]:
data0 = pd.read_excel('../data_sources/Old/GSE147507/GSE147507.xlsx')

In [ ]:
for column in data0.columns: print(column)

In [5]:
data0.index = list(data0['Unnamed: 0'])
data0.drop(columns = ['Unnamed: 0']+[_ for _ in list(data0.columns) if _[6] in ['3','4','8']],
          inplace = True)
data0.drop(columns = list(data0.loc[:,'Series9_NHBE_IAV_1':'Series9_NHBE_IAV_4'])+list(data0.loc[:,'Series9_NHBE_IAVdNS1_1':'Series9_NHBE_IAVdNS1_4']), 
          inplace = True )

In [ ]:
for column in data0.columns: print(column)

In [6]:
data = data0.copy()

In [7]:
#NORMALIZACION

for GAPDH,Columna in zip(list(data.loc['GAPDH']),list(data.columns)) :
    data[Columna] = data[Columna]/GAPDH

In [8]:
series_ids = ["1", "2", "5", "6", "7", "9", "15", "16"]
Series = {}

In [9]:
for id in series_ids:
    Series[id] = data.loc[:,list(S for S in list(data.columns) if f"Series{id}_" in S[:9])].copy()

In [11]:
Total_Gen = len(data.index)
for key in Series.keys():
    measurement_types = dict.fromkeys(map(lambda x: x[:-2], Series[key]))
    for typ in measurement_types.keys():
        Series[key][f"Promedio_{typ}"] = list(sts.mean(Series[key].filter(like=typ, axis=1).iloc[fila]) for fila in range (Total_Gen))
        

In [ ]:
data

In [ ]:
#fold-change
for key in Series.keys():
    # Remove else False
    measurement_types = [x for x in Series[key] if "Promedio" in x]
    if key != "15":
        mock = Series[key].filter(regex="^Promedio.*Mock$").columns[0]
    else:
        mock = Series[key].filter(regex="Promedio_Series15_HealthyLungBiopsy").columns[0]
    measurement_types.remove(mock)

    if len(measurement_types) > 1:
        for typ in measurement_types:
            name = typ.split(f"_Series{key}_")[1].replace("_", " ")
            Series[key][f"fold-change Series{key} {name}"] = fold_change(Series[key][mock], Series[key][typ])
    else:
        typ = measurement_types[0]
        Series[key][f"fold-change Series{key}"] = fold_change(Series[key][mock], Series[key][typ])


In [ ]:
Series["9"]

In [ ]:
#CONJUNTO TOTAL
for key in Series.keys():
    data = pd.concat([data, Series[key].filter(like="Promedio_Series", axis=1)], axis=1)
    data = pd.concat([data, Series[key].filter(like="fold-change", axis=1)], axis=1)


In [ ]:
#DF fold-change
fold_change_data = data.filter(like='fold-change')

In [ ]:
fold_change_data

In [ ]:
missing_genes = Genes-set(fold_change_data.index)
fold_change_data = pd.dataFrame(fold_change_data, index=list(fold_change_data.index)+list(missing_genes), columns=fold_change_data.columns)